# Attribute Descriptors

A descriptor is a class that implements a dynamic protocol consisting of the __get__,
__set__, and __delete__ methods.

### LineItem Take #3: A Simple Descriptor
As we said in the introduction, a class implementing a __get__, a __set__, or a
__delete__ method is a descriptor. You use a descriptor by declaring instances of it
as class attributes of another class.

In [2]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)
    def __get__(self, instance, owner):
        return instance.__dict__[self.storage_name]

In [3]:
class House:
    rooms = Quantity('number_of_rooms')

In [4]:
def __get__(self, instance, owner):
    if instance is None:
        return self
    else:
        return instance.__dict__[self.storage_name]

In [5]:
class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

LineItem Take #4: Automatic Naming of Storage Attributes

In [8]:
class Quantity2:
    def __set_name__(self, owner, name):
        self.storage_name = name
    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

# no __get__ needed
class LineItem:
    weight = Quantity2()
    price = Quantity2()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price
